In [1]:
import pandas as pd
import json
import os

folder_path = "Sample Data"
file_name = "fiscal_calendar.csv"
file_path = os.path.join(folder_path, file_name)

with open(file_path, "r", encoding="utf-8") as f:
    df = pd.read_csv(f)
print(df)
# Convert column names dynamically
df.columns = (
    df.columns.str.strip()  # Remove leading/trailing spaces
    .str.lower()            # Convert to lowercase
    .str.replace(" ", "_")  # Replace spaces with underscores
    .str.replace(r"[^\w\s]", "", regex=True)  # Remove special characters
)

            Date  Fiscal Year  Fiscal Quarter  Fiscal Month  Calendar Year  \
0     2024-02-01         2024               1             2           2024   
1     2024-02-02         2024               1             2           2024   
2     2024-02-03         2024               1             2           2024   
3     2024-02-04         2024               1             2           2024   
4     2024-02-05         2024               1             2           2024   
...          ...          ...             ...           ...            ...   
9857  2051-01-27         2050               4             1           2051   
9858  2051-01-28         2050               4             1           2051   
9859  2051-01-29         2050               4             1           2051   
9860  2051-01-30         2050               4             1           2051   
9861  2051-01-31         2050               4             1           2051   

      Calendar Quarter  Calendar Month  Week of Year Day of Wee

In [3]:
import snowflake.connector
import os
import requests
PASSWORD = os.getenv('SNOWSQL_PWD')
print(PASSWORD)

5eWxWv4EvyCkhkY


In [ ]:
try:
    ctx = snowflake.connector.connect(
        user='ABHINAVSHARMA2002',
        password=PASSWORD,
        account='qraojwa-yg67137'
    )
    cs = ctx.cursor()
    try:
        cs.execute("CREATE WAREHOUSE IF NOT EXISTS task_1_warehouse_mg")
        cs.execute("CREATE DATABASE IF NOT EXISTS testdb_mg")
        cs.execute("USE DATABASE testdb_mg")
        cs.execute("CREATE SCHEMA IF NOT EXISTS task_1_mg")
        cs.execute("USE WAREHOUSE task_1_warehouse_mg")
        cs.execute("USE SCHEMA task_1_mg")
        runQueries(cs,df)
    except Exception as e:
        print(f"Error: {e}") 
    finally:
        cs.close()
        ctx.close()
except Exception as e:
    print(f"Error: {e}")

In [5]:
def runQueries(cs,df):    
    # Rename specific columns
    #df["customer_id"] = df["customer_id"].astype(str)
    df = df.astype(str).apply(lambda col: col.map(lambda x: None if pd.isna(x) or x == "" or str(x).strip().lower() == "nan" else x))

    # Generate CREATE TABLE statement dynamically
    table_name = "article_table_pandas_3"
    columns = ", ".join([f'{col} STRING' for col in df.columns])
    create_table_sql = f'CREATE OR REPLACE TABLE {table_name} ({columns})'
    cs.execute(create_table_sql)

    # Insert data into Snowflake
    for _, row in df.iterrows():
        placeholders = ", ".join(["%s"] * len(row))
        insert_sql = f'INSERT INTO {table_name} ({", ".join(df.columns)}) VALUES ({placeholders})'
        cs.execute(insert_sql, tuple(row))